# Fluence Demo

To run this fluence demo you need to step through each cell in turn by pressing the keys `Shift + Enter`. Make sure the very first cell has completed running before you run any further cells. Using `Cell > Run All` does not work with VPython at this current time.

In [ ]:
import ivisual as vis

In [ ]:
from copy import copy

import numpy as np
from ipywidgets import interact, fixed
from IPython.display import display, Markdown, clear_output

In [ ]:
try:
    fluence_scene.delete()
except:
    pass

fluence_scene = vis.idisplay(
    title=' ', background=(0.94, 0.94, 0.94),
    autoscale=False, userzoom=False, userspin=False
)
fluence_scene.select()

max_num_photons = 300

photon_x0 = np.random.normal(-45, 1, max_num_photons)
photon_y0 = np.random.normal(0, 1, max_num_photons)
photon_z0 = np.random.normal(0, 1, max_num_photons)

photon_initial_positions = [
    (photon_x0[i], photon_y0[i], photon_z0[i])
    for i in range(max_num_photons)
]

photons = [
    vis.sphere(
        pos=photon_initial_position, color=(0.4,0.4,0.9), 
        radius=0.5, material=vis.materials.plastic, opacity=1)
    for photon_initial_position in photon_initial_positions
]

In [ ]:
photon_velocity_x = np.random.normal(5, 1, max_num_photons)
photon_velocity_y = np.random.normal(0, 1, max_num_photons)
photon_velocity_z = np.random.normal(0, 1, max_num_photons)

intersect_time = -photon_x0 / photon_velocity_x
intersect_y = photon_y0 + photon_velocity_y * intersect_time
intersect_z = photon_z0 + photon_velocity_z * intersect_time

In [ ]:
numpy_photon_initial_positions = np.array(photon_initial_positions)
numpy_photon_velocities = np.vstack([photon_velocity_x, photon_velocity_y, photon_velocity_z]).T

In [ ]:
fluence_region = vis.box(
    pos=vis.vector(0,0,0),
    size=vis.vector(0.1, 10, 10),
    color=(0.9,0.9,0.2),
    opacity=1,
    material=vis.materials.rough)

def set_fluence_region_area(area):
    edge = np.sqrt(area)
    fluence_region.size = vis.vector(0.1, edge, edge)

In [ ]:
current_photon_number = copy(max_num_photons)
current_time = 0
current_fluence_area = 100

def fluence_interaction(time=current_time, number_of_photons=current_photon_number, fluence_area=current_fluence_area):
    global current_photon_number, current_time, current_fluence_area    
    
    current_positions = numpy_photon_initial_positions + numpy_photon_velocities * time    
    for i, photon in enumerate(photons):
        photon.pos = vis.vector(*current_positions[i])
    
    if number_of_photons != current_photon_number:
        for i in range(0, number_of_photons):
            photons[i].visible = True

        for i in range(number_of_photons, max_num_photons):
            photons[i].visible = False
            
    fluence_side = np.sqrt(fluence_area)
    colour_red = (
        (time > intersect_time) &
        (np.abs(intersect_y) < fluence_side/2) &
        (np.abs(intersect_z) < fluence_side/2)
    )    
    
    if (time != current_time) | (fluence_area != current_fluence_area):
        for i in np.where(colour_red)[0]:
            photons[i].color = (0.9,0.4,0.4)

        colour_blue = np.invert(colour_red)
        for i in np.where(colour_blue)[0]:
            photons[i].color = (0.4,0.4,0.9)
            
    if fluence_area != current_fluence_area:
        set_fluence_region_area(fluence_area)
        
    counted_photons = np.sum(colour_red[0:number_of_photons])
    display(Markdown('Number of photons passed through region = {}'.format(counted_photons)))
    display(Markdown('Fluence = Number of photons / Area of region = {}'.format(counted_photons/fluence_area)))
            
    current_photon_number = number_of_photons
    current_time = time
    current_fluence_area = fluence_area

In [ ]:
display(Markdown('# Fluence demo'))
display(Markdown('Right click and drag to change the camera angle'))
fluence_scene.background = (0.94, 0.94, 0.94)
fluence_scene.userspin = True
fluence_scene.userzoom = False
fluence_scene.autoscale = False
fluence_scene.range = (45,20,20)
display(fluence_scene)
display(fluence_scene)

In [ ]:
interact(
    fluence_interaction, 
    time=[0.0, 20.0], 
    number_of_photons=[0,max_num_photons],
    fluence_area=[10, 1000]
);